# Single family (SFH) vs. Multi family (MFH) homes

## Hypothesis
Our zoning code strongly discourages multi-family homes. Are city government representatives fairly representing their voters, or their districts?

In [1]:
import pandas as pd
import geopandas as geo_pd
import dvc.api

In [3]:
with dvc.api.open("data/gis/wardsprecincts2022_poly.zip") as f:
    wards_df = geo_pd.read_file(f)

FileMissingError: Can't find 'wardsprecincts2022_poly.zip' neither locally nor on remote